The worflow:

for each frame:
    mask cars appearing on frame at instant t (maybe use dilation)
    extract edges (evaluate this two changing the order)
    get Lines filtered by certain range of angles (get horizontal and vertical, but separately) (maybe explore this for lane markings)
Once lines are saved:
        get histogram of orientations
        get histogram of lengths
        remove outliers
map all endpoints of lines to pre defined lengths
use them to train a MLP.
Test
    

In [1]:
import cv2
import numpy as np
import torchvision.transforms as T

from sklearn.neighbors import KernelDensity
#from sklearn.grid_search import GridSearchCV
#from sklearn.cross_validation import LeaveOneOut
from objectDetection import *
from carMasking import *
from imgFeatureUtils import *

%matplotlib inline

forExt = DLForegroundExtractor(512)
videoPath = "/home/hector/speed/speed_DS/v1.mp4"

In [2]:

forExt = TraditionalForegroundExtractor()
cap = cv2.VideoCapture("/home/hector/speed/speed_DS/v1.mp4")
if (cap.isOpened()== False): 
  print("Error opening video stream or file")

while(cap.isOpened()):
  ret, frame = cap.read()
  if ret == True:
    grayFrame = convertImg2Grayscale(frame)
    foregroundFrame = forExt.maskCarsAtFrame(grayFrame)
    edgeMap = getEdgeMap(foregroundFrame,100,200)
    lines = getLinesFromEdges(edgeMap,(-20,10), 10, 0)
    empty = np.zeros((edgeMap.shape[0],edgeMap.shape[1],3))
    empty[:,:,0] = edgeMap
    linesImg = drawLinesOnImg(empty, lines)
    cv2.imshow('Frame',linesImg)
    if cv2.waitKey(25) & 0xFF == ord('q'):
      break
  else: 
    break
cap.release()
cv2.destroyAllWindows()


In [9]:
yolo = objectDetector((512,512),
                       weightPath="yolov3/weights/yolov3.weights",
                       modelDef="yolov3/config/yolov3.cfg")
viz = objectVisualizer(classPath="yolov3/data/coco.names")
tools = objectFocusTools((512,512),yolo)
linesDataset = []
cap = cv2.VideoCapture("/home/hector/speed/speed_DS/v5.avi")
featureSet = []

if (cap.isOpened()== False): 
  print("Error opening video stream or file")
while(cap.isOpened()):
  ret, frame = cap.read()
  if ret == True:
    frame = cv2.resize(frame, (512,512))
    grayFrame = convertImg2Grayscale(frame)
#(-60,-40) vertical, 
    lines, edgeMap = tools.getObjectLinesAtFrame(frame,grayFrame,(-60,-40))
    lineDist = getLinesEuclideanDist(lines)
    thetas = getLinesThetas(lines)
    featurePoints = list(zip(lineDist,                  #distance
                       [line[0][0] for line in lines],  #y pos
                       [line[0][1] for line in lines])) #pos
    print(thetas)
    #print(featurePoints)
    featureSet.append(featurePoints)

    empty = np.zeros((edgeMap.shape[0],edgeMap.shape[1],3))
    empty[:,:,0] = edgeMap
    #frameObjs = viz.drawBoxes(frame,detections)
    linesImg = drawLinesOnImg(empty, lines)
    cv2.imshow('Frame',linesImg)
    if cv2.waitKey(25) & 0xFF == ord('q'):
      break
  else: 
    break
featureSet = flatten(featureSet)
#print("final feature set")
#print(featureSet)
cap.release()
cv2.destroyAllWindows()

2809621944]
[-0.8960553845713439, -0.866302262552679, -0.982793723247329, -1.0303768265243125, -1.0121970114513341, -0.7853981633974483, -0.8076167287241673, -0.8198672643969563, -0.8198672643969563, -0.8584387287005677, -0.8156919233162234, -0.8176450458327023, -0.7583777142101835, -0.7354397676755056, -0.8156919233162234, -0.8884797719201486]
[-0.8139618212362083, -0.8960553845713439, -0.7853981633974483, -0.8058034940839864, -0.8960553845713439, -0.8156919233162234, -0.8315192794119595, -0.724250000748147, -0.8097835725701669, -0.7419472680059175, -0.8288490587889791, -0.8210972760767723, -0.7551044034786732]
[-0.9944211062037129, -0.7853981633974483, -0.8818719385800352, -0.9380474917927134, -0.7853981633974483, -0.8198672643969563, -0.866302262552679, -0.7853981633974483, -0.7509290623979403, -0.8760580505981934, -0.7853981633974483, -0.7621465405869854, -0.8233546085857627, -0.8042638494191191, -0.8478169733934057, -0.7531512809621944]
[-0.8884797719201486, -0.9467732738181398, -

In [7]:
featureSetNumpy = np.array(featureSet)
print(featureSetNumpy.shape)

(104, 3)
